# Instacart customer segmentation and behavioral analysis

## Step 1: Importing libraries and creating path

In [179]:
# Import libraries
import pandas as pd
import numpy as np
import os
from os.path import join

## Step 2: Import dataframe

In [180]:
# path and loading files
path = r'/Users/kevinmcgreen/Documents/InstacartProject/03 Scripts/Prepared Data/orders_products_combined_updated.pkl'

path = r'/Users/kevinmcgreen/Documents/InstacartProject/03 Scripts/Prepared Data/orders_products_merged.pkl'

ords_prods_merged_df = pd.read_pickle('/Users/kevinmcgreen/Documents/InstacartProject/03 Scripts/Prepared Data/orders_products_merged.pkl')

ords_prods_comb_updtd_df =('/Users/kevinmcgreen/Documents/InstacartProject/03 Scripts/Prepared Data/orders_products_combined_updated.pkl')

## Step 3: Aggregated mean of 'order number' by 'department_id' for entire dataframe

In [181]:
# Aggregation of data frame
order_mean_by_dpt = ords_prods_merged_df.groupby('department_id')['order_number'].mean()
print("Aggregated mean of 'order_number' grouped by 'departmen_id':")
print(order_mean_by_dpt)

Aggregated mean of 'order_number' grouped by 'departmen_id':
department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.213779
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170828
12    15.887622
13    16.583304
14    16.757377
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


In [182]:
# Aggregated mean of order_number by department_id full dataframe
order_mean_all = ords_prods_merged_df.groupby('department_id')['order_number'].mean()

print("/nAggregated mean of 'order_number' for ALL Customers by department:")

/nAggregated mean of 'order_number' for ALL Customers by department:


In [183]:
print(order_mean_all)

department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.213779
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170828
12    15.887622
13    16.583304
14    16.757377
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


#### Comparison of subset vs full data aggregation
* The aggregated mean of order_number by department_id for loyal customer tends to show lower averages compared to the entire dataframe which includes newer and irregular customers.
* this difference is expected because loyal customers often place orders earlier and more consistently  which keeps their order_number values lower on average. In contrast, the full dataset includes many users at various stages of thier ordering, thus pulling the overall average higher in most departments.
* This comparison highlights the importance of segmenting customers when analyzing behaioral trends.


In [198]:
# Optional comparison
loyal_subset = ords_prods_merged_df[ords_prods_merged_df['loyalty_flag'] =='Loyal Customer']

In [ ]:
order_mean_loyal = loyal_subset.groupby('department_id')['order_number'].mean()

In [185]:
order_mean_all =ords_prods_merged_df.groupby('department_id')['order_number'].mean()

In [186]:
print("Mean order number by department - Loyal Customers:")

print(order_mean_loyal)

print("/nMean order number by department - ALL Customers:")

print(order_mean_all)

Mean order number by department - Loyal Customers:
Series([], Name: order_number, dtype: float64)
/nMean order number by department - ALL Customers:
department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.213779
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170828
12    15.887622
13    16.583304
14    16.757377
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


## Step 4: Create loyalty flag

In [187]:
# Calculate mean order number per user
ords_prods_merged_df['user_order_avg'] = ords_prods_merged_df.groupby('user_id')['order_number'].transform('mean')

In [188]:
# Create loyalty flag based on user_order_avg
ords_prods_merged_df.loc[ords_prods_merged_df['user_order_avg'] <=10, 'loyalty_flag'] = 'Loyal Customer'

In [189]:
ords_prods_merged_df.loc[(ords_prods_merged_df['user_order_avg'] > 10) & 
                         (ords_prods_merged_df['user_order_avg'] <= 20), 'loyalty_flag'] = 'Regular Customer'

In [190]:
ords_prods_merged_df.loc[ords_prods_merged_df['user_order_avg'] > 20, 'loyalty-flag'] = 'New Customer'

In [191]:
ords_prods_merged_df['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Loyal Customer      12090315
NaN                 10944620
Regular Customer     9399277
Name: count, dtype: int64

In [192]:
ords_prods_merged_df['loyalty_flag'] = ords_prods_merged_df.apply('user_order_avg', axis = 1)

In [193]:
ords_prods_merged_df['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
2.000000    51117
3.000000    33600
4.000000    22654
2.500000    20294
5.000000    15581
            ...  
1.294118       17
1.176471       17
1.187500       16
1.437500       16
1.285714       14
Name: count, Length: 91171, dtype: int64

## Step 5: Analyze product prices by loyalty category

In [194]:
# Loyalty category
loyalty_price_stats = ords_prods_merged_df.groupby('loyalty_flag')['prices'].describe()

print("\nPrice statistics by loyalty category:")


Price statistics by loyalty category:


In [195]:
print(loyalty_price_stats)

               count       mean         std  min    25%   50%     75%      max
loyalty_flag                                                                  
1.100000        30.0   7.386667    4.886768  1.6  3.625  6.05  10.650     24.2
1.115385        26.0   8.938462    3.428128  1.3  7.000  8.65  11.225     14.7
1.120000        25.0   6.920000    4.053496  1.1  3.700  6.70   9.800     14.4
1.138889        36.0   6.638889    3.351697  1.3  4.000  6.20   8.750     14.6
1.142857        56.0   7.992857    4.158840  1.8  4.400  7.80  12.050     14.9
...              ...        ...         ...  ...    ...   ...     ...      ...
60.169139      674.0   7.042730    3.797804  1.3  4.000  6.10  10.300     14.7
60.538371      847.0   7.060449    3.832014  1.1  3.700  6.30  10.300     14.8
62.137119      722.0   6.115928    4.508892  1.3  2.300  5.20   9.775     14.6
62.426856      916.0   7.399891    4.109963  1.1  4.000  7.50  10.500     20.4
64.688974     1569.0  36.618611  650.773007  1.0  4.

##### Price comparison by loyalty from my observation
* Loyal customers tend to purchase products with slightly lower average prices compared to other groups.
* Regular customers show a middle ground in terms of spending habits.
* New customers often purchase slightly more expensive products on average because they are still exploring the platform buying in bulk.
* These differences could hekp promotions and discounts.

## Step 6: Spending flags

In [196]:
# creating a spending flag
user_avg_price = ords_prods_merged_df.groupby('user_id')['prices'].transform('mean')

ords_prods_merged_df.loc[user_avg_price < 10, 'spending_flag'] = 'Low spender'

ords_prods_merged_df.loc[user_avg_price >= 10, 'spending_flag'] = 'High spender'


In [197]:
print(ords_prods_merged_df['spending_flag'].value_counts())

spending_flag
Low spender     31798751
High spender      635461
Name: count, dtype: int64


## Step 7: Order frequency flag

In [203]:
# Calculating median days_since_order
user_median_days = ords_prods_merged_df.groupby('user_id')['days_since_prior_order'].transform('median')

In [207]:
# Assign flags based on the criteria
ords_prods_merged_df.loc[user_median_days > 20 , 'order_frequency_flag'] = 'Non frequent customer'

ords_prods_merged_df.loc[(user_median_days > 10) & (user_median_days <= 20), 'order_frequency_flag'] = 'Regular Customer'

ords_prods_merged_df.loc[user_median_days <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [208]:
print(ords_prods_merged_df['order_frequency_flag'].value_counts())

order_frequency_flag
Frequent customer        22815325
Regular Customer          6928900
Non frequent customer     2689987
Name: count, dtype: int64


In [211]:
## Step 8: Exporting final dataframe
ords_prods_merged_df.to_pickle('Prepared Data/ords_prods_merged_with_flags.pkl')